In [2]:
from skimage import io, transform, color
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
from dataLoader import OrganoidDataset
from torch.utils import data
import numpy as np
import sys
import pandas as pd
from imageio import imread
from PIL import Image
import os

from dataLoader import OrganoidDataset
#from conv_model import SimpleConvNet
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
figure_path = '../milestoneReport/figures/'
path = '../data/CS231n_Tim_Shan_example_data/'
label_path = '../data/'

In [5]:
wellA1 = pd.read_csv('../data/well_summary_A1_e0891BSA_all.csv')
wellA2 = pd.read_csv(label_path+'well_summary_A2.csv')
wellA3 = pd.read_csv(label_path+'well_summary_A3.csv')
wellA4 = pd.read_csv(label_path+'well_summary_A4.csv')
wellC1 = pd.read_csv(label_path+'well_summary_C1.csv')

In [6]:
wellA1['condition'] = 'A1'
wellA2['condition'] = 'A2'
wellA3['condition'] = 'A3'
wellA4['condition'] = 'A4'
wellC1['condition'] = 'C1'

In [8]:
wellA1.shape, wellA2.shape,wellA3.shape,wellA4.shape,wellC1.shape

((67200, 7), (50400, 7), (10796, 7), (33600, 7), (44800, 7))

In [10]:
wellA1.head(2)

,well id,day,median pixel intensity,mw_area shape,hyst1 area,hyst2 area,condition
0,0,0,125.0,7830,512,418,A1
1,1,0,125.0,8265,3044,3008,A1


In [9]:
wellA2.head(2)

,well id,day,median pixel intensity,mw_area shape,hyst1 area,hyst2 area,condition
0,0,0,130.0,32396,11222,10702,A2
1,1,0,131.0,9500,43,0,A2


In [7]:
wellC1.head(2)

,well id,day,median pixel intensity,mw_area shape,hyst1 area,hyst2 area,condition
0,0,0,117.0,7224,139,141,C1
1,1,0,118.0,7308,422,388,C1


In [8]:
139596/14.0

9971.142857142857

In [11]:
wellA1.groupby(by='day')['well id'].count()

day
0     4800
1     4800
2     4800
3     4800
4     4800
5     4800
6     4800
7     4800
8     4800
9     4800
10    4800
11    4800
12    4800
13    4800
Name: well id, dtype: int64

In [12]:
wellA2.groupby(by='day')['well id'].count()

day
0     3600
1     3600
2     3600
3     3600
4     3600
5     3600
6     3600
7     3600
8     3600
9     3600
10    3600
11    3600
12    3600
13    3600
Name: well id, dtype: int64

In [9]:
wellA3.groupby(by='day')['well id'].count()

day
3    3600
5    3600
6    3596
Name: well id, dtype: int64

In [10]:
wellC1.groupby(by='day')['well id'].count()

day
0     3200
1     3200
2     3200
3     3200
4     3200
5     3200
6     3200
7     3200
8     3200
9     3200
10    3200
11    3200
12    3200
13    3200
Name: well id, dtype: int64

In [11]:
wellA2.groupby(by='day')['hyst2 area'].sum()

day
0     11118030
1      9515283
2      9699242
3     10188886
4      9387332
5     11237400
6     10763021
7     12244466
8     11280029
9     11625314
10    12835175
11    13009691
12    13693762
13    13596488
Name: hyst2 area, dtype: int64

In [24]:
wellC1.groupby(by='day')['hyst2 area'].sum()

day
0     1259622
1      928593
2      929989
3     1128199
4     1661649
5     2061019
6     1971614
7     1753965
8     1991609
9     2784164
10    2098197
11    2164090
12    2155446
13    2357182
Name: hyst2 area, dtype: int64

In [48]:
label = pd.concat([wellA1,wellA2,wellC1],sort=False)
new_columns = ['well_id','day','median_pixel_intensity','mw_area_shape','hyst1_area','hyst2_area','condition']
label.columns = new_columns

In [49]:
label.head(2)

,well_id,day,median_pixel_intensity,mw_area_shape,hyst1_area,hyst2_area,condition
0,0,0,125.0,7830,512,418,A1
1,1,0,125.0,8265,3044,3008,A1


In [50]:
def get_image_name(well_id,day_id):
    image_name = 'well'+str(well_id).zfill(4)+'_day'+str(day_id).zfill(2)+'_well.png'
    return image_name
def get_well_label(well_id):
    return str(well_id).zfill(4)

In [52]:
label['image_name'] = label[['well_id','day']].apply(lambda x: get_image_name(*x),axis=1)
label['image_name'] = 'well_'+label['condition']+'/'+label['image_name']
label['well_label'] = label['well_id'].apply(lambda x: get_well_label(x))
label['has_cell'] = np.int64(label['hyst2_area'] > 0)

In [53]:
label.head(3)

,well_id,day,median_pixel_intensity,mw_area_shape,hyst1_area,hyst2_area,condition,image_name,well_label,has_cell
0,0,0,125.0,7830,512,418,A1,well_A1/well0000_day00_well.png,0000,1
1,1,0,125.0,8265,3044,3008,A1,well_A1/well0001_day00_well.png,0001,1
2,2,0,127.0,8280,30,0,A1,well_A1/well0002_day00_well.png,0002,0


In [54]:
label.shape

(162400, 10)

In [55]:
corrected_labels = label.query("mw_area_shape < 32000")

In [56]:
corrected_labels.shape

(154322, 10)

In [57]:
def get_day_description(corrected_labels,day):
    df = corrected_labels.query("day == %d"%day)[['condition','well_id','day','well_label','image_name','has_cell']]
    day_suffix = '_' + str(day)
    df = df.rename(columns={'day':'day'+day_suffix, 'has_cell':'has_cell'+day_suffix,'image_name':'image_name'+day_suffix})
    return df

In [58]:
# day2wells = get_day_description(corrected_labels,2)
# day8wells = get_day_description(corrected_labels,8)

In [59]:
# day2wells.shape,day8wells.shape

In [60]:
# day2wells.merge(day8wells,on=['well_id','well_label','condition']).shape

In [61]:
prev = get_day_description(corrected_labels,0)
for day in range(1,14):
    #print(day)
    curr = get_day_description(corrected_labels,day)
    #print(curr.shape[0])
    prev = prev.merge(curr,on=['well_id','well_label','condition'])

In [62]:
prev.shape

(8143, 45)

In [63]:
merged_days = prev.copy()

In [64]:
#shuffle dataframe
merged_days = merged_days.sample(frac=1).reset_index(drop=True)

In [65]:
train,validation,test = np.split(merged_days, [int(.8 * len(merged_days)), int(.9 * len(merged_days))])

In [66]:
train.shape,validation.shape,test.shape

((6514, 45), (814, 45), (815, 45))

In [67]:
train.head(4)

,condition,well_id,day_0,well_label,image_name_0,has_cell_0,day_1,image_name_1,has_cell_1,day_2,...,has_cell_10,day_11,image_name_11,has_cell_11,day_12,image_name_12,has_cell_12,day_13,image_name_13,has_cell_13
0,A2,495,0,0495,well_A2/well0495_day00_well.png,1,1,well_A2/well0495_day01_well.png,1,2,...,1,11,well_A2/well0495_day11_well.png,1,12,well_A2/well0495_day12_well.png,1,13,well_A2/well0495_day13_well.png,1
1,A1,588,0,0588,well_A1/well0588_day00_well.png,1,1,well_A1/well0588_day01_well.png,1,2,...,1,11,well_A1/well0588_day11_well.png,1,12,well_A1/well0588_day12_well.png,1,13,well_A1/well0588_day13_well.png,1
2,A2,1803,0,1803,well_A2/well1803_day00_well.png,1,1,well_A2/well1803_day01_well.png,1,2,...,1,11,well_A2/well1803_day11_well.png,1,12,well_A2/well1803_day12_well.png,1,13,well_A2/well1803_day13_well.png,1
3,C1,2459,0,2459,well_C1/well2459_day00_well.png,0,1,well_C1/well2459_day01_well.png,0,2,...,0,11,well_C1/well2459_day11_well.png,0,12,well_C1/well2459_day12_well.png,0,13,well_C1/well2459_day13_well.png,0


In [68]:
prev.to_csv("A1_A2_C1_filtered_all_v2.csv")
train.to_csv("A1_A2_C1_filtered_train_v2.csv")
validation.to_csv("A1_A2_C1_filtered_validation_v2.csv")
test.to_csv("A1_A2_C1_filtered_test_v2.csv")

# normalization

In [33]:
def update_mean(x, x_bar_prev, n):
  return x_bar_prev + (x - x_bar_prev)/n

def update_sum_square_diff(x, ssd_prev, x_bar_prev, x_bar_curr):
  return ssd_prev + (x - x_bar_prev)*(x - x_bar_curr)

In [69]:
def get_mean_std(image_names,path = label_path):
    #image_names = merged_days['image_name_1']
    x_bar_curr = 0.0
    x_bar_prev = 0.0
    ssd_curr = 0.0
    n = 0
    for index in range(len(image_names)):
        img_name = image_names[index]
        img_loc = os.path.join(path, img_name)
      # skimage.io.imread returns a numpy array
        image = io.imread(img_loc)
      # convert to grey scale
        image = color.rgb2gray(image)
        for i, x in np.ndenumerate(image):
            n = n + 1
            x_bar_prev = x_bar_curr
            x_bar_curr = update_mean(x, x_bar_prev, n)
            ssd_curr = update_sum_square_diff(x, ssd_curr, x_bar_prev, x_bar_curr)
    return x_bar_curr, np.sqrt(ssd_curr/n)

In [ ]:
print("yes")

In [70]:
#day8_mean, day8_std = get_mean_std(merged_days['image_name_8'])
day2_mean, day2_std = get_mean_std(merged_days['image_name_2'])

In [71]:
day2_mean, day2_std

(0.4942364496334061, 0.16092630656356338)

In [ ]:
mean_sd_dict = {}

In [35]:
day = 0

In [38]:
'image_name_%d'%day

'image_name_0'

In [39]:
path

'../data/CS231n_Tim_Shan_example_data/'

In [37]:
def get_mean_std_for_all_days(max_day_range=14):
    for day in range(max_day_range):
        mean, std = get_mean_std(merged_days['image_name_%d'%day])
        mean_sd_dict[day] = [mean, std]
    return mean_sd_dict

In [ ]:
mean_sd_dict[2] = [day2_mean, np.sqrt(day2_std)]
mean_sd_dict[8] = [day8_mean, np.sqrt(day8_std)]

In [ ]:
condition 

In [42]:
label_path + 'well_'      

'../data/well_'